<a href="https://colab.research.google.com/github/PaulaDCV/Scraping_Job_Vacancies/blob/main/Job_Vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports/Setup



In [ ]:
!pip install requests
!pip install BeautifulSoup4
!pip install scrapfly-sdk
!pip install selenium
!pip install selenium-stealth

In [ ]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import time
from scrapfly import ScrapflyClient, ScrapeConfig
import pandas as pd


# Indeed

### URL Template


## Scraping

In [ ]:
def url_generation_indeed(position, location):
  url = "https://uk.indeed.com/jobs?q="+position+"&l="+location
  return url

In [ ]:
def request_listings_indeed(url):
  client = ScrapflyClient(key=secret)
  result = client.scrape(ScrapeConfig(
      url= url,
      asp=True,
      # ^ enable Anti Scraping Protection
  ))
  html_text = result.content
  soup = BeautifulSoup(html_text, "html5lib")
  return soup

In [ ]:
url = url_generation_indeed("data","london")
print("URL:"+url)
soup = request_listings_indeed(url)


URL:https://uk.indeed.com/jobs?q=data&l=london


### unused code

In [ ]:
def request_listings_old(url):
  #Change User Agent to avoid being blocked
  headers = {
      "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
  }
  html_text = requests.get(url, headers = headers).text
  soup = BeautifulSoup(html_text, "html5lib")
  return soup

## Processing Step

In [ ]:
#Extract individual info of each job
def extract_job_info_indeed(job):
  jobs = []
  title_element = job.find("h2", class_="jobTitle")
  jobs.append( title_element.text)

  href = title_element.find("a",attrs= {"role": "button"}).get("href","Not Available")
  jobs.append(href)

  location = job.find("div", attrs={"data-testid": "text-location" })
  jobs.append(location.text)

  company = job.find("span", attrs ={ "data-testid" : "company-name"} )
  jobs.append(company.text)

  jobs.append("indeed")

  return jobs

In [ ]:
#Extract every job posting into a list
def extract_jobs_indeed(results,position_search, location_search):
  #Create dataframe
  df = pd.DataFrame(columns = ['Title', 'Url', 'Location','Company', 'Site',
                               'Position Search','Location Search'])
  #Get Jobs
  jobs = results.find_all("td", class_="resultContent")
  #Extract info of each job
  for job in jobs:
    job_processed = extract_job_info_indeed(job)
    job_processed.extend([position_search,location_search])
    df.loc[len(df.index)] = job_processed
  return df

In [ ]:
extract_jobs_indeed(soup)

,Title,Url,Location,Company,Site
0,Data & Insights Associate,/rc/clk?jk=15c12930da592997&fccid=4819693c833d...,London,Takeda Pharmaceutical,indeed
1,Data Surveillance Information Officer - London,/rc/clk?jk=fdef229953a94280&fccid=38b7144572a9...,Canary Wharf E14,NHS Professionals,indeed
2,Data Manager,/rc/clk?jk=0128cd40dbc3f2cc&fccid=7691ebb71b24...,Royal Victoria Station,The Newcastle upon Tyne Hospitals NHS Foundati...,indeed
3,Graduate Trainee – NatWest Markets - Quantitat...,/rc/clk?jk=5d9284117fe72392&fccid=4a3a2b1eb3ec...,London,NatWest Markets,indeed
4,Junior Data Analyst,/rc/clk?jk=0501ff18c4042280&fccid=5ba44a57b6e0...,London,Qualient Solutions,indeed
5,"Operations and Technology, Graduate Data Scien...",/rc/clk?jk=cb8e8f360bbb3c5a&fccid=5bcd1ef0a7f4...,London,Citi,indeed
6,Health Data Quality Clerk (15 hours per week),/rc/clk?jk=d719051d28041d0a&fccid=7691ebb71b24...,London SE1,Guy's and St Thomas' NHS Foundation Trust,indeed
7,Data Migration BA,/rc/clk?jk=c2895d936f0a1084&fccid=f511c48ddba0...,Hybrid remote in London,Janus Henderson,indeed
8,Data Entry Administrator,/rc/clk?jk=29a9363427598303&fccid=4a7f522d3544...,London SE1,Informa Connect,indeed
9,Junior Commercial Analyst,/rc/clk?jk=033d98756f37e634&fccid=3083ff994f73...,London,University College London,indeed


In [ ]:
def run_indeed(positions,location):
    df = pd.DataFrame(columns = ['Title', 'Url', 'Location','Company', 'Site',
                               'Position Search','Location Search'])

  #Search job listings for every positions
  for poistion in positions:

    #Generate the URL for the job search
    url = url_generation_indeed("data","london")
    print("URL:"+url)

    #Request data from the generated url
    soup = request_listings_indeed(url)

    #If there are jobs in the search extract individual job info
    if len(soup)<0:

      temp = extract_jobs_indeed(soup,position, location)
      print("Indeed jobs for " + position+ "in London" + scraped")

      #Add job listings to the main list
      pd.concat([df, temp], axis=0)
  return df

# Glassdoor
-Unfinished

### URL Template




1.   https://www.glassdoor.com/Job/london-data-scientist-jobs-SRCH_IL.0,6_IC2671300_KO7,21.htm?includeNoSalaryJobs=true
2.   https://www.glassdoor.com/Job/brighton-england-uk-data-scientist-jobs-SRCH_IL.0,19_IC2680716_KO20,34.htm





* Location: You can specify a location using the &location= parameter. For example, &location=New+York would search for jobs in New York.

* Job Title/Keyword: You can specify a job title or keyword using the &sc.keyword= parameter. For example, &sc.keyword=software+engineer would search for software engineering jobs.

* Company: You can specify a company using the &sc.company= parameter. For example, &sc.company=Google would search for jobs at Google.

* Job Type: You can specify the type of job using the &jobType= parameter. For example, &jobType=fulltime would search for full-time jobs.



https://www.glassdoor.com/Job/england-uk-data-jobs-SRCH_IL.0,10_IS7287_KO11,15.htm

In [ ]:
def url_generation_glassdoor(position, location):
  url = "https://www.glassdoor.com/Job/"+location+"-"+position+"-jobs-SRCH_IL.0,10_IS7287_KO11,15.html"
  print(url)
  return url

In [ ]:
def request_listings_glassdoor(url):
  client = ScrapflyClient(key=secret)
  result = client.scrape(ScrapeConfig(
      url= url,
      asp=True,
      render_js = True

      # ^ enable Anti Scraping Protection
  ))

  html_text = result.content
  soup = BeautifulSoup(html_text, "html5lib")
  return soup

In [ ]:
url = url_generation_glassdoor("data","England-UK")
soup = request_listings_glassdoor(u
                                  rl)
print(soup)


https://www.glassdoor.com/Job/England-UK-data-jobs-SRCH_IL.0,10_IS7287_KO11,15.html
<!DOCTYPE html>
<html class="fonts-loaded-initial fonts-loaded-full" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>9,036 data Jobs in England, November 2023 | Glassdoor</title><meta content="Glassdoor" property="og:site_name"/><meta content="website" property="og:type"/><meta content="https://www.glassdoor.com/assets/images/meta/logo-og.png" property="og:image"/><meta content="9,036 data Jobs in England, November 2023" property="og:title"/><meta content="Search Data jobs in England with company ratings &amp; salaries. 9,036 open jobs for Data in England." property="og:description"/><meta content="https://www.glassdoor.com/Job/england-data-jobs-SRCH_IL.0,7_IS7287_KO8,12.htm" property="og:url"/><meta content="Search Data jobs in England with company ratings &amp; salarie

In [ ]:
#Extract individual info of each job

GlassDoorData = {"Job Postings":["ul","class","job"],
                 "Title":["a","id","job-title"],
                 "Href":["a",""],
                 "Location":["div","id","job-location"],
                 "Company":["span","class","employer-Name"]

                 }

def extract_job_info_glassdoor(job):
  jobs = []
  title_element = job.find("h2", class_="jobTitle")
  jobs.append( title_element.text)
  jobs.append(title_element.href)

  location = job.find("div", attrs={"id": "job-location" })
  jobs.append(location.text)

  company = job.find("span", attrs ={ "class" : "employer-Name"} )
  jobs.append(company.text)

  jobs.append("Indeed")
  return jobs

In [ ]:
#Extract every job posting into a list
def extract_jobs_glassdoor(results):
  #Create dataframe
  df = pd.DataFrame(columns = ['Title', 'Url', 'Location','Company', 'Site'])
  #Get Jobs

  jobs = results.find_all("script")
  print(jobs)
  #Extract info of each job
  for job in jobs:
    print(jobs)
    job_processed = extract_job_info_glassdoor(job)
    df.loc[len(df.index)] = job_processed
  return df

## Scraping attempt 2


In [ ]:
import os
import re
import subprocess
import requests

# The deb files we need to install
deb_files_startstwith = [
    "chromium-codecs-ffmpeg-extra_",
    "chromium-codecs-ffmpeg_",
    "chromium-browser_",
    "chromium-chromedriver_"
]

def get_latest_version() -> str:
    # A request to security.ubuntu.com for getting latest version of chromium-browser
    # e.g. "112.0.5615.49-0ubuntu0.18.04.1_amd64.deb"
    url = "http://security.ubuntu.com/ubuntu/pool/universe/c/chromium-browser/"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Find latest version
    pattern = '<a\shref="chromium\-browser_([^"]+.ubuntu0\.18\.04\.1_amd64\.deb)'
    latest_version_search = re.search(pattern, text)
    if latest_version_search:
        latest_version = latest_version_search.group(1)
    else:
        raise Exception("Can not find latest version!")
    return latest_version

def download(latest_version: str, quiet: bool):
    deb_files = []
    for deb_file in deb_files_startstwith:
              deb_files.append(deb_file + latest_version)

    for deb_file in deb_files:
        url = f"http://security.ubuntu.com/ubuntu/pool/universe/c/chromium-browser/{deb_file}"

        # Download deb file
        if quiet:
            command = f"wget -q -O /content/{deb_file} {url}"
        else:
            command = f"wget -O /content/{deb_file} {url}"
        print(f"Downloading: {deb_file}")
        # os.system(command)
        !$command

        # Install deb file
        if quiet:
            command = f"apt-get install /content/{deb_file} >> apt.log"
        else:
            command = f"apt-get install /content/{deb_file}"
        print(f"Installing: {deb_file}\n")
        # os.system(command)
        !$command

        # Delete deb file from disk
        os.remove(f"/content/{deb_file}")

def check_chromium_installation():
    try:
        subprocess.call(["chromium-browser"])
        print("Chromium installation successfull.")
    except FileNotFoundError:
        print("Chromium Installation Failed!")

def install_selenium_package(quiet: bool):
    if quiet:
        !pip install selenium -qq >> pip.log
    else:
        !pip install selenium

def main(quiet: bool):
    # Get the latest version of chromium-browser for ubuntu 18.04
    latest_version = get_latest_version()
    # Download and install chromium-browser for ubuntu 20.04
    download(latest_version, quiet)
    # Check if installation succesfull
    check_chromium_installation()
    # Finally install selenium package
    install_selenium_package(quiet)

if __name__ == '__main__':
    quiet = True # verboseness of wget and apt
    main(quiet)

Downloading: chromium-codecs-ffmpeg-extra_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-codecs-ffmpeg-extra_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-codecs-ffmpeg_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-codecs-ffmpeg_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-browser_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-browser_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-chromedriver_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-chromedriver_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Chromium installation successfull.


In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("start-maximized")

wd = webdriver.Chrome( options=chrome_options)
wd.get("https://www.google.com")

print(f"Page title: {wd.title}")

Page title: Google


In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("start-maximized")

# Adding argument to disable the AutomationControlled flag
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# Exclude the collection of enable-automation switches
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Turn-off userAutomationExtension
chrome_options.add_experimental_option("useAutomationExtension", False)


wd = webdriver.Chrome( options=chrome_options)
url = url_generation_glassdoor("data","England-UK")
result = wd.get(url)
print(f"Page title: {wd.title}")
html_text = wd.page_source
soup = BeautifulSoup(html_text, "html5lib")
soup

In [ ]:
from selenium import webdriver
from selenium_stealth import stealth
import time

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument('--headless')
options.add_argument('--no-sandbox')

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)


stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

url = url_generation_glassdoor("data","England-UK")
result = driver.get(url)
time.sleep(3)
print(f"Page title: {driver.title}")
html_text = driver.page_source
soup = BeautifulSoup(html_text, "html5lib")


https://www.glassdoor.com/Job/England-UK-data-jobs-SRCH_IL.0,10_IS7287_KO11,15.html
Page title: 6,301 data Jobs in England, November 2023 | Glassdoor


<ul class="contentMenu__contentMenuStyles__contentMenuContainer" data-test="content-menu" id="ContentNav"><li class="contentMenu__contentMenuStyles__contentMenuItem" data-test="site-header-community"><div class="contentMenu__contentMenuStyles__itemAnimation"><div><svg height="1536" preserveAspectRatio="xMidYMid meet" style="width: 100%; height: 100%; transform: translate3d(0px, 0px, 0px); content-visibility: visible;" viewBox="0 0 2000 1536" width="2000" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><defs><clipPath id="__lottie_element_2"><rect height="1536" width="2000" x="0" y="0"></rect></clipPath><clipPath id="__lottie_element_5"><path d="M0,0 L1236,0 L1236,1236 L0,1236z"></path></clipPath><clipPath id="__lottie_element_21"><path d="M0,0 L1236,0 L1236,1236 L0,1236z"></path></clipPath><clipPath id="__lottie_element_37"><path d="M0,0 L1236,0 L1236,1236 L0,1236z"></path></clipPath><clipPath id="__lottie_element_53"><path d="M0,0 L384,0 L384,384 L0,384z"

In [ ]:
#Extract individual info of each job

GlassDoorData = {"Job Postings":["ul","class","job"],
                 "Title":["a","id","job-title"],
                 "Href":["a",""],
                 "Location":["div","id","job-location"],
                 "Company":["span","class","employer-Name"]

                 }

def extract_job_info_glassdoor(job):
  jobs = []
  title_element = job.find("h2", class_="jobTitle")
  jobs.append( title_element.text)
  jobs.append(title_element.href)

  location = job.find("div", attrs={"id": "job-location" })
  jobs.append(location.text)

  company = job.find("span", attrs ={ "class" : "employer-Name"} )
  jobs.append(company.text)

  jobs.append("Indeed")
  return jobs

In [ ]:
#Extract every job posting into a list
def extract_jobs_glassdoor(results):
  #Create dataframe
  df = pd.DataFrame(columns = ['Title', 'Url', 'Location','Company', 'Site'])
  #Get Jobs

  jobs = results.find_all("div",attributes ={"class":"JobsList"})
  print(results)
  #Extract info of each job
  for job in jobs:
    print(jobs)
    job_processed = extract_job_info_glassdoor(job)
    df.loc[len(df.index)] = job_processed
  return df

In [ ]:
import re
import json
script_text = soup.find_all("script")
print(script_text)
# get javascript object inside the script
model_data = re.search(r"modelData = ({.*?});", script_text, flags=re.S)
model_data = model_data.group(1)

# "convert" the javascript object to json-valid object
model_data = re.sub(
    r"^\s*([^:\s]+):", r'"\1":', model_data.replace("'", '"'), flags=re.M
)

# json decode the object
model_data = json.loads(model_data)

# print the data
print(model_data["name"])
print(model_data["thumbnailUrl"])
print(model_data["account"])


[<script async="" src="https://s.pinimg.com/ct/lib/main.74d80534.js"></script>, <script async="" src="https://snap.licdn.com/li.lms-analytics/insight.old.min.js"></script>, <script async="" data-id="C5TG9KG68TKST8M2I6O0" src="https://analytics.tiktok.com/i18n/pixel/static/main.MTdjYzNiZDU2MA.js" type="text/javascript"></script>, <script async="" id="pdst-capture" src="https://cdn.pdst.fm/ping.min.js"></script>, <script async="" src="https://connect.facebook.net/signals/config/1499075380345177?v=2.9.138&amp;r=stable&amp;domain=www.glassdoor.com"></script>, <script async="" src="//connect.facebook.net/en_US/fbevents.js"></script>, <script async="" src="https://www.redditstatic.com/ads/pixel.js" type="text/javascript"></script>, <script async="" src="https://s.pinimg.com/ct/core.js" type="text/javascript"></script>, <script async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js" type="text/javascript"></script>, <script async="" src="https://static.ads-twitter.com/uwt.js" ty

TypeError: ignored

# CSV File

In [ ]:
def main(position, location, csv):

  indeed = run_indeed(position,location)
  #glassdoor  = run_glassdoor(position,location)

  jobs_processed = indeed #+glassdoor
  jobs_processed.to_csv(csv+".csv",index=False)

main(["data"],"london", "DataLondon")